In [ ]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np
from datetime import date

# Working Dir.
os.chdir('/Users/fogellmcmuffin/Documents/ra/team_discussions/AI/')
os.environ['OPENAI_API_KEY'] = 'sk-proj-BBrbzGnsFwklndehaTC1T3BlbkFJEGPQt0QfhWkwp9ePuxaK'

In [ ]:
####################
## Model Settings ##
####################

# Calling for OpenAI client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    organization='org-WLFAmqjnKmywM0wd6loMyGJq',    # RA_WORK
    project='proj_vOr6WeeCFk5IjZLCdksFLWUd',    # IRPD_CODING
)

# Model Settings
MODEL = 'gpt-4o-2024-05-13'
TEMPERATURE = 0
MAX_TOKENS = 1300
TOP_P = 1
FREQUENCY_PENALTY = 0
PRESENCE_PENALTY = 0

In [ ]:
###############
## Functions ##
###############

## General/Help functions
def file_to_string(file_path):  # File read-to-string functions
  with open(file_path, 'r') as file:
    k = file.read()
  return str(k)


def write_file(file_path, file_write):  # File write function
  with open(file_path, 'w') as file:
    file.write(file_write)     


def test_info(test, data_name):  # Function to get test info
  info = str(
    'ChatGPT Model Information:' + '\n' +
    'format: OpenAI API' + '\n' +
    'model: ' + str(MODEL) + '\n' +
    'temperature: ' + str(TEMPERATURE) + '\n' +
    'max tokens: ' + str(MAX_TOKENS) + '\n' +
    'top p: ' + str(TOP_P) + '\n' +
    'frequency penalty: ' + str(FREQUENCY_PENALTY) + '\n' +
    'presence penalty: ' + str(PRESENCE_PENALTY) + '\n\n' +
    'Test Information:' + '\n' + 
    'test: ' + test + '\n' +
    'data: ' + data_name + '\n' +
    'date: ' + str(date.today()) + '\n'
  )
  return info


def get_test_dir(test_type='test'):  # Function to get test directory
  if test_type == 'test':
    test_path = 'output/'
    tests = [i for i in os.listdir(test_path) if i.startswith('test')]
    
    test_numbers = [int(k[5:]) for k in tests]
    new_test_number = max(test_numbers) + 1
    
    test_dir = f"test_{new_test_number}"
  elif test_type == 'subtest':
    subtest_path = 'output/_subtests/'
    
    subtests = [int(k) for k in os.listdir(subtest_path) if k.isdigit()]
    new_test_number = max(subtests) + 1
    
    test_dir = str(new_test_number)
  return test_dir


## Prompt request functions
def GPT_response(sys, user):  # Simple GPT response function
  # Requesting chat completion
  response = client.chat.completions.create(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    frequency_penalty=FREQUENCY_PENALTY,
    presence_penalty=PRESENCE_PENALTY,
    messages=[
      {"role": "system", "content": str(sys)},
      {"role": "user", "content": str(user)}
    ]
  )
  output = response.choices[0].message.content  # GPT response var
  
  return output


def stage_1_output(treatment, test_type='test'):  # Stage 1 function: Creating categories
  # Prompts
  sys_ucoop = file_to_string(f'prompts/ucoop/{treatment}/sys_1_{treatment}_ucoop.md')
  sys_udef = file_to_string(f'prompts/udef/{treatment}/sys_1_{treatment}_udef.md')
  window_prompts = [['ucoop', sys_ucoop], ['udef', sys_udef]]
  
  # Making test directory
  if test_type == 'test':
    test = get_test_dir()
    test_dir = (os.path.join('output/', test))
    info_path = os.path.join(test_dir, f't{test[5:]}_test_info.txt')  # Test info path
  elif test_type == 'subtest':
    test = get_test_dir(test_type='subtest')
    test_dir = (os.path.join('output/_subtests/', test))
    info_path = os.path.join(test_dir, f'{test}__subtest_info.txt')  # Test info path
  
  os.makedirs(test_dir, exist_ok=False)
  
  # Test info
  info = test_info(test, f'RAsum_{treatment}_ucoop.csv & RAsum_{treatment}_udef.csv')
  write_file(info_path, info)
  
  # Creating stage 1 windows
  for i in window_prompts:
    stage_dir = os.path.join(test_dir, f'stage_1_{i[0]}')
    os.makedirs(stage_dir, exist_ok=False)  # Stage directory
    
    # GPT request output
    output = GPT_response(i[1], i[2])
    
    # Creating paths for prompts & GPT response
    sys_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_1_{i[0]}_sys_prmpt.txt')
    user_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_1_{i[0]}_user_prmpt.txt')
    response_path = os.path.join(stage_dir, f't{test[5:]}_stg_1_{i[0]}_response.txt')
    
    # Writing .txt files for prompts & GPT response
    write_file(sys_prmpt_path, i[1])
    write_file(user_prmpt_path, i[2])
    write_file(response_path, str(output))
  
  return print("Stage 1 Complete")


def stage_2_output(treatment, test, df_ucoop, df_udef): # Stage 2 function: Assigning categories to individual summaries (recursive)
  # utf-8 encoding system prompt
  sys_ucoop = file_to_string(f'code/prompts/ucoop/{treatment}/sys_2_{treatment}_ucoop.md')
  sys_udef = file_to_string(f'code/prompts/udef/{treatment}/sys_2_{treatment}_udef.md')
  window_prompts = [['ucoop', sys_ucoop, df_ucoop], ['udef', sys_udef, df_udef]]
  
  test_dir = f'output/{test}/'    # Test directory
  
  for i in window_prompts:
    stage_dir = os.path.join(test_dir, f'stage_2_{i[0]}')
    os.makedirs(stage_dir, exist_ok=False)  # Stage directory
    
    # Writing .txt system prompt (the same for every request)
    sys = i[1]    # System prompt for ucoop or udef data
    sys_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_2_{i[0]}_sys_prmpt.txt')
    write_file(sys_prmpt_path, sys)
    
    # Prompt & Response paths
    prompt_path = os.path.join(stage_dir, 'prompts')
    response_path = os.path.join(stage_dir, 'responses')
    os.makedirs(prompt_path, exist_ok=True)
    os.makedirs(response_path, exist_ok=True)
    
    # Requesting chat completion for each row
    df = i[2]   # Test data for ucoop or udef data
    for k in range(len(df)):
      row = df.iloc[k].to_dict()  # Creating a dictionary for each indv. row
      
      output = GPT_response(sys, str(row))  # GPT request output
      
      # Creating paths for prompts & GPT responses using window_numbers
      window_number = row['window_number']
      user_prmpt_path = os.path.join(prompt_path, f't{test[5:]}_{window_number}_user_prmpt.txt')
      output_path = os.path.join(response_path, f't{test[5:]}_{window_number}_response.txt')
      
      # Writing .txt files for prompts & GPT response
      write_file(user_prmpt_path, str(row))
      write_file(output_path, str(output))
  
  return print("Stage 2 Complete")

In [ ]:
##############
## Requests ##
##############

## Preliminaries
test = 'test_10'
window_type = 'ucoop'
treatment = 'noise'
summary_data = True
ucoop_test_data = pd.read_csv(f'test_data/RAsum_{treatment}_ucoop.csv')
udef_test_data = pd.read_csv(f'test_data/RAsum_{treatment}_udef.csv')
ucoop_test_data = ucoop_test_data[:20]
ucoop_test_data['window_number'] = ucoop_test_data['window_number'].astype(int)
udef_test_data = udef_test_data[:20]
udef_test_data['window_number'] = udef_test_data['window_number'].astype(int)

In [ ]:
## Sub-test
sys, user, df_name = get_testing_info(window_type, treatment, 1, summary_data)
test_GPT(sys, user, df_name)

In [ ]:
## Stage 1
stage_1_output(treatment, test)

In [ ]:
## Stage 1 Refinement
stage_1r_output(treatment, test)

In [ ]:
## Stage 2
stage_2_output(treatment, test, ucoop_test_data, udef_test_data)

In [ ]:
test_data = pd.read_csv(f'test_data/RAsum_noise_ucoop.csv')

shuffled_test_data = test_data.sample(frac=1, random_state=42)
test_data_sets = np.array_split(shuffled_test_data, 8)

shuffled_test_sets = [subset.to_dict(orient='records') for subset in test_data_sets]
print(str(shuffled_test_sets[2]))